## GPT 환각 검증

> GPT가 생성한 검색 키워드가 실제 Spotify 결과와 **맥락적으로 일치하는지(환각 검증)** 확인 


---

### ① **규칙 기반 검증 (Taxonomy 매핑 검증)**

**검증 단위** : GPT가 만든 “검색 키워드 자체”
**비교 기준** : Taxonomy가 정의한 규칙셋(예상값)
**역할**: GPT가 제시한 키워드가 이론적으로 맞는 방향인지 확인

**과정:**

* GPT가 생성한 키워드에 대응되는 음악이 `Soundscape Taxonomy`의 규칙 범위에 부합하는지 확인한다.
* 예를 들어, `library × quiet × focus` 환경인데 GPT가 생성한 키워드가 “EDM high energy”라면 Taxonomy상 focus의 예상 BPM은 75–90인데 EDM은 120 이상 → “규칙 위반(환각 가능)”
* 검증 변수: `bpm_range`, `energy_range`, `mood`, `genre_primary`, `vocal`

**의의:**
GPT의 언어적 연상 결과가 Soundscape 규칙과 **음향적 속성 일관성**을 가지는지 판단할 수 있다.

---

### ② **결과-쿼리 유사도 검증 (Semantic Alignment Test)**

**과정:**

* GPT 키워드(`prompt_query`)와 Spotify에서 검색된 트랙의 메타데이터(`track_name`, `artist`, `genre`, `description`)를 임베딩하여 **코사인 유사도**를 계산한다.
* 일정 임계값(예: 0.65 이하)이면 “맥락 불일치(환각)”로 분류한다.
* Python 구현 시 `sentence-transformers`나 `openai.embeddings`를 활용 가능.

**의의:**
텍스트 수준에서 GPT의 의도(설명)와 실제 Spotify 콘텐츠가 **의미적으로 일치**하는지 정량화할 수 있다.

*참고*
메타데이터 수집:

/v1/tracks → popularity, explicit, name, preview_url, artists

/v1/artists → genres

주의: 일부 추천·관련 아티스트/카테고리 계열도 제한됨. 우리는 사용하지 않음. 

---

### ③ **음악 속성 프록시 변수 기반 검증: Empirical Result Validation**

*참고*

/v1/audio-features(tempo/energy/valence/instrumentalness…)가 신규앱에 제한됨 평가 설계를 메타데이터/장르/제목 플래그/프리뷰URL 기반 **프록시**로 판정해야함

**검증 단위** : GPT 키워드로 Spotify에서 실제 검색된 결과물
비교 기준: Taxonomy 규칙이 아니라, 결과 트랙의 메타데이터 특성(=프록시 피처)

프록시 변수 예시:
| 직접 피처        | 프록시 변수                                       | 설명                               |
| ------------ | -------------------------------------------- | -------------------------------- |
| tempo(BPM)   | `track_duration_ms`                          | 평균적으로 BPM과 상관관계 있음               |
| energy       | `popularity`, `genre`, `explicit`            | 활발한 곡일수록 popularity↑, explicit↑  |
| mood/valence | `playlist_name`, `album_name`, `description` | “chill”, “happy”, “sad” 등 키워드 존재 |
| vocal 여부     | `track_name`, `artist`                       | instrumental/feat. 등의 표현         |

**과정**:

GPT 키워드로 검색된 Spotify 결과 N개를 수집
프록시 변수들을 추출하고 Taxonomy 목표값(예: calm, focus 등)에 맞는지 통계적으로 비교
예: focus인데 playlist_name이 “EDM Party”, explicit 비율이 80%면 → “맥락 불일치(환각)”

*프록시 변수 매핑*

| 검증 목적(예상규칙)                                  | 프록시 변수                                                                                             | 필드/출처                                      | 호출/스코프                                                          |
| -------------------------------------------- | -------------------------------------------------------------------------------------------------- | ------------------------------------------ | --------------------------------------------------------------- |
| BPM/에너지 저중/중저(예: `focus`: BPM 75–90, 에너지 낮음) | **`duration_ms` 분포**(극단적 초단편·초장편 비율), **제목 토큰**(“remix/extended/live/club”)                        | Track 객체(`duration_ms`), Track/Playlist 이름 | Get Track(무스코프), Search(무스코프) ([Spotify for Developers][1])     |
| 장르 일치성(예: `lo-fi`, `ambient`, `piano`)       | **아티스트 `genres`** → 트랙에 간접 매핑                                                                      | Artist 객체(`genres`)                        | Get Artist(무스코프) ([Spotify for Developers][2])                  |
| 무보컬/담백함(예: `focus`, `reading`)               | **트랙명 키워드**(instrumental, piano, acoustic, ambient), **플레이리스트명/설명 키워드**(study, focus, lofi, chill) | Track/Playlist 이름·설명                       | Search, Get Playlist(공개는 무스코프) ([Spotify for Developers][3])    |
| 과한 자극 회피(예: `relax`, `sleep`)                | **`explicit` 비율↓**, **플레이리스트 설명 키워드**(calm, sleep, soothe)                                         | Track(`explicit`), Playlist 설명             | Get Track, Get Playlist(공개) ([Spotify for Developers][4])       |
| 밝기/정서(예: `relax`는 ‘sad’ 과다이면 불일치)            | **제목/플레이리스트 설명 감성 토큰**(happy/sad/chill/party/angry 등)                                              | Track/Playlist 이름·설명                       | Search, Get Playlist(공개) ([Spotify for Developers][3])          |
| 대중성으로 에너지 간접추정                               | **`popularity` 평균/분포**(고에너지/힙합 파티 등은 인기 상승 경향)                                                     | Track(`popularity`)                        | Get Track, 일부 라이브러리 엔드포인트에도 설명 있음 ([Spotify for Developers][5]) |
| 소음 환경 적합성(이동/야외=자극 내성↑)                      | **가용국가 수(`available_markets`)**, **미리듣기 존재율(`preview_url`)**로 유통폭/콘텐츠 타입 감지(약함)                    | Track(`available_markets`, `preview_url`)  | Get Track(무스코프) ([Spotify for Developers][6])                   |
| 최신성/시대감(예: `active`는 최신 팝 비중↑)               | **`album.release_date`**로 신곡 비중                                                                    | Track→Album 경유                             | Get Track/Album Tracks(무스코프) ([Spotify for Developers][7])      |
| 장르가 직접 없을 때 우회                               | **아티스트 조회로 장르 보강**                                                                                 | Artist(`genres`)                           | Get Artist(무스코프) + 일반적 우회 관행 ([Spotify for Developers][2])      |

[1]: https://developer.spotify.com/documentation/web-api/reference/get-track?utm_source=chatgpt.com "Get Track"
[2]: https://developer.spotify.com/documentation/web-api/reference/get-an-artist?utm_source=chatgpt.com "Spotify Web API Reference - Get an Artist"
[3]: https://developer.spotify.com/documentation/web-api/reference/search?utm_source=chatgpt.com "Web API Reference | Spotify for Developers"
[4]: https://developer.spotify.com/documentation/web-api/reference/get-the-users-currently-playing-track?utm_source=chatgpt.com "Get Currently Playing Track"
[5]: https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks?utm_source=chatgpt.com "Get User's Saved Tracks"
[6]: https://developer.spotify.com/documentation/web-api/reference/get-users-top-artists-and-tracks?utm_source=chatgpt.com "Get User's Top Items"
[7]: https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks?utm_source=chatgpt.com "Get Album Tracks - Web API Reference"



---

### + 사용 가능한 Web API만 정리

Search GET /v1/search (type=track, market, limit, offset)

Tracks GET /v1/tracks, GET /v1/tracks?ids=

Artists GET /v1/artists, GET /v1/artists?ids=

(지양) Recommendations/Related Artists/Audio Features/Audio Analysis: 신규 앱 제한/Deprecated → 사용 안 함

---
② **결과-쿼리 유사도 검증 (Semantic Alignment Test)**

OpenAI: pip install openai (또는 최신 SDK 이름에 맞게)

로컬: pip install sentence-transformers scikit-learn requests pandas numpy

로컬사용함

In [18]:
import os, re, time, json, requests
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [ ]:
#토큰발급
'''import requests

CLIENT_ID = "2caa74d47f2b40449441b09fbaec95ed"
CLIENT_SECRET = "8cf5431e5d65428bb133d902f8a71344"

r = requests.post(
    "https://accounts.spotify.com/api/token",
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials"},
    auth=(CLIENT_ID, CLIENT_SECRET)
)
print(r.json()["access_token"])'''


BQDfsee4vtdxaHksFOKbiGUwcN1j1YdQx4JP7X56oagK4ysQMDa2LdDIPjn3QbJImmLEBuf2gyLYU03YeXXSkIrxq3zLl2EIOtQ_Lye0HqyrASy6eEw0ZG9mPKaLj-8Zaheni9gqzGQ


In [25]:
# ====== 0. 파라미터 ======
TOP_K_TRACKS = 10          # 트랙 상위 K
TOP_K_PLAYLISTS = 5        # 플레이리스트 상위 K
TIMEOUT = 20               # 초
SIM_THRESHOLD = 0.60
K_FOR_PRECISION = 10
OUT_CSV = "semantic_alignment_report_local.csv"

In [26]:
# ====== 1. Spotify API: 토큰 & 헤더 유틸 ======
def clean_bearer(raw: str) -> str:
    if raw is None:
        raise ValueError("Access token is None")
    s = raw.replace("\r", "").replace("\n", "").replace("\t", "").strip()
    s = re.sub(r"[^\x20-\x7E]", "", s)  # ASCII printable만 유지
    return s

def ascii_header_value(v: str) -> str:
    v = (v or "").strip()
    v = v.encode("latin-1", "ignore").decode("latin-1")
    return v

In [28]:
#토큰 자동발급함수
import requests
from base64 import b64encode

CLIENT_ID = "2caa74d47f2b40449441b09fbaec95ed"
CLIENT_SECRET = "8cf5431e5d65428bb133d902f8a71344"

def get_spotify_token(client_id: str, client_secret: str) -> str:
    r = requests.post(
        "https://accounts.spotify.com/api/token",
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data={"grant_type": "client_credentials"},
        auth=(client_id, client_secret),
        timeout=15,
    )
    r.raise_for_status()
    return clean_bearer(r.json()["access_token"])

SPOTIFY_BEARER = get_spotify_token(CLIENT_ID, CLIENT_SECRET)

def spotify_headers():
    return {
        "Authorization": ascii_header_value(f"Bearer {SPOTIFY_BEARER}"),
        "User-Agent": "python-requests SoundscapeSemanticEval"
    }



In [29]:
# ====== 2. 로컬 임베딩 모델 ======
print("SentenceTransformer 모델 로드 중...")
EMBEDDER = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("로드 완료")

def embed_texts(texts):
    return EMBEDDER.encode(texts, normalize_embeddings=True)

def cosine_sim(a, b):
    a = a / (np.linalg.norm(a) + 1e-9)
    b = b / (np.linalg.norm(b) + 1e-9)
    return float(np.dot(a, b))


SentenceTransformer 모델 로드 중...
로드 완료


In [30]:
# ====== 3. Spotify 검색 함수 ======

SEARCH_URL = "https://api.spotify.com/v1/search"

def spotify_search(query: str, search_type="track", limit=10, timeout=TIMEOUT, retry=2):
    global SPOTIFY_BEARER
    params = {"q": query, "type": search_type, "limit": int(limit)}
    for attempt in range(retry + 1):
        try:
            r = requests.get(SEARCH_URL, headers=spotify_headers(), params=params, timeout=timeout)
            if r.status_code == 401 and attempt < retry:
                # 토큰 만료 → 갱신 후 재시도
                SPOTIFY_BEARER = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
                continue
            if r.status_code == 429 and attempt < retry:
                wait_sec = int(r.headers.get("Retry-After", "1"))
                time.sleep(wait_sec)
                continue
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            if attempt >= retry:
                print("HTTPError:", e, "Body:", getattr(e.response, "text", "")[:500])
                raise
        except UnicodeEncodeError:
            print("Header encoding error. 토큰을 clean_bearer()로 정리했는지 확인하세요.")
            raise
    return {}

def normalize_text(s):
    return re.sub(r"\s+", " ", (s or "").strip())

def build_track_doc(track_obj):
    name = (track_obj.get("name") or "").strip()
    artists = ", ".join([(a or {}).get("name", "") for a in (track_obj.get("artists") or [])]).strip()
    album = ((track_obj.get("album") or {}).get("name") or "").strip()
    text = " ".join([t for t in [name, f"by {artists}" if artists else "", f"album {album}" if album else ""] if t])
    return normalize_text(text)

def build_playlist_doc(playlist_obj):
    name = (playlist_obj.get("name") or "").strip()
    desc = (playlist_obj.get("description") or "").strip()
    owner = ((playlist_obj.get("owner") or {}).get("display_name") or "").strip()
    text = " ".join([t for t in [name, desc, f"by {owner}" if owner else ""] if t])
    return normalize_text(text)

def collect_candidate_docs(gpt_query):
    docs = []
    # 트랙
    tr = spotify_search(gpt_query, "track", TOP_K_TRACKS)
    track_items = (tr.get("tracks") or {}).get("items") or []
    for item in track_items:
        if not isinstance(item, dict): 
            continue
        tid = item.get("id")
        if not tid:
            continue
        doc = build_track_doc(item)
        if doc:
            docs.append((tid, "track", doc))
    # 플레이리스트
    pr = spotify_search(gpt_query, "playlist", TOP_K_PLAYLISTS)
    playlist_items = (pr.get("playlists") or {}).get("items") or []
    for item in playlist_items:
        if not isinstance(item, dict):
            continue
        pid = item.get("id")
        if not pid:
            continue
        doc = build_playlist_doc(item)
        if doc:
            docs.append((pid, "playlist", doc))
    return docs


In [ ]:
# ====== 4. 유사도 기반 평가 ======
def evaluate_one_case(query_id, gpt_query, intent_context,
                      sim_threshold=SIM_THRESHOLD, k_for_precision=K_FOR_PRECISION):
    full_query = normalize_text(f"{gpt_query}. context: {intent_context}")
    docs = collect_candidate_docs(gpt_query)

    if not docs:
        return {
            "query_id": query_id,
            "gpt_query": gpt_query,
            "intent_context": intent_context,
            "mean_sim": np.nan,
            "best_sim": np.nan,
            "p_at_k": 0.0,
            "hallucinated": True,
            "top_examples": "[]"
        }

    q_vec = embed_texts([full_query])[0]
    d_texts = [d[2] for d in docs]
    d_vecs = embed_texts(d_texts)

    sims = [cosine_sim(q_vec, dv) for dv in d_vecs]
    order = np.argsort(sims)[::-1]
    sims_sorted = [sims[i] for i in order]
    docs_sorted = [docs[i] for i in order]

    K = min(k_for_precision, len(sims_sorted))
    p_at_k = float(np.mean([1.0 if sims_sorted[i] >= sim_threshold else 0.0 for i in range(K)]))
    mean_sim = float(np.mean(sims_sorted[:K]))
    best_sim = float(sims_sorted[0])

    hallucinated = (mean_sim < sim_threshold) or (p_at_k < 0.3) or (best_sim < (sim_threshold + 0.02))

    top3 = []
    for i in range(min(3, len(docs_sorted))):
        top3.append({
            "rank": i + 1,
            "id": docs_sorted[i][0],
            "type": docs_sorted[i][1],
            "sim": round(sims_sorted[i], 4),
            "text": docs_sorted[i][2][:200]
        })

    return {
        "query_id": query_id,
        "gpt_query": gpt_query,
        "intent_context": intent_context,
        "mean_sim": round(mean_sim, 4),
        "best_sim": round(best_sim, 4),
        "p_at_k": round(p_at_k, 3),
        "hallucinated": bool(hallucinated),
        "top_examples": json.dumps(top3, ensure_ascii=False)
    }

In [ ]:
# ====== 5. 배치 평가 & 리포트 ======
def run_semantic_eval(cases, sim_threshold=SIM_THRESHOLD, k_for_precision=K_FOR_PRECISION):
    results = []
    for qid, q, ctx in cases:
        res = evaluate_one_case(qid, q, ctx, sim_threshold, k_for_precision)
        results.append(res)
        print(f"[{qid}] {q} -> mean={res['mean_sim']} best={res['best_sim']} p@k={res['p_at_k']} hallucinated={res['hallucinated']}")
        time.sleep(0.3)
    df = pd.DataFrame(results)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\n📄 결과 저장: {OUT_CSV}")
    return df

# ====== 6. 실행 예시 ======
if __name__ == "__main__":
    cases = [
        ("Q001", "lofi study beats focus", "library, quiet, focus"),
        ("Q002", "energetic workout pop edm", "gym, loud, active"),
        ("Q003", "calm sleep ambient", "home, night, sleep")
    ]
    df = run_semantic_eval(cases)
    print(df)

    #규칙엔진 출력이 없어서 6번 “실행 예시”로 샘플 케이스를 넣어 검증만 돌린 상태
    #규칙엔진 출력값나오면 케이스만 교체해서 검증
    

[Q001] lofi study beats focus -> mean=0.3038 best=0.5231 p@k=0.0 hallucinated=True
[Q002] energetic workout pop edm -> mean=0.4766 best=0.6863 p@k=0.2 hallucinated=True
[Q003] calm sleep ambient -> mean=0.6164 best=0.7344 p@k=0.4 hallucinated=False

📄 결과 저장: semantic_alignment_report_local.csv
  query_id                  gpt_query         intent_context  mean_sim  \
0     Q001     lofi study beats focus  library, quiet, focus    0.3038   
1     Q002  energetic workout pop edm      gym, loud, active    0.4766   
2     Q003         calm sleep ambient     home, night, sleep    0.6164   

   best_sim  p_at_k  hallucinated  \
0    0.5231     0.0          True   
1    0.6863     0.2          True   
2    0.7344     0.4         False   

                                        top_examples  
0  [{"rank": 1, "id": "4hs8q3NwLAMKuQsKJ2uP4n", "...  
1  [{"rank": 1, "id": "40d3ceIUf7R7xzVWerX6LX", "...  
2  [{"rank": 1, "id": "3ndo89a4dJ87sjRqKiDIlI", "...  


In [33]:
#평가케이스 확장 (총 25건 쿼리후보 수동생성)

if __name__ == "__main__":
    cases = [
    # ---- focus (집중) ----
    ("Q001", "lofi study beats focus", "library, quiet, focus"),
    ("Q002", "instrumental concentration playlist", "co-warking, quiet, focus"),
    ("Q003", "chill cafe background for work", "cafe, moderate, focus"),
    ("Q004", "deep focus ambient sound", "home, silent, focus"),

    # ---- relax (휴식) ----
    ("Q005", "chillout jazz relax evening", "cafe, moderate, relax"),
    ("Q006", "soft acoustic coffee break", "park, moderate, relax"),
    ("Q007", "relaxing piano background", "home, quiet, relax"),
    ("Q008", "smooth rnb unwind", "co-warking, quiet, relax"),

    # ---- sleep (수면) ----
    ("Q009", "calm sleep ambient", "home, silent, sleep"),
    ("Q010", "rain sounds for sleeping", "home, quiet, sleep"),
    ("Q011", "night meditation deep sleep", "library, silent, sleep"),

    # ---- active (활력) ----
    ("Q012", "energetic workout pop edm", "gym, loud, active"),
    ("Q013", "morning running motivation", "park, loud, active"),
    ("Q014", "cardio boost playlist", "moving, loud, active"),
    ("Q015", "driving upbeat hits", "moving, moderate, active"),

    # ---- anger (분노) ----
    ("Q016", "heavy metal anger release", "gym, very_loud, anger"),
    ("Q017", "hard rock stress relief", "moving, loud, anger"),
    ("Q018", "rage playlist aggressive beats", "home, loud, anger"),

    # ---- consolation (위로) ----
    ("Q019", "comforting acoustic ballads", "home, quiet, consolation"),
    ("Q020", "warm indie songs for comfort", "cafe, moderate, consolation"),
    ("Q021", "soothing piano for heartbreak", "library, silent, consolation"),

    # ---- neutral (미설정) ----
    ("Q022", "daily mix chill pop", "home, moderate, neutral"),
    ("Q023", "morning commute playlist", "moving, moderate, neutral"),
    ("Q024", "background pop hits", "co-warking, moderate, neutral"),
    ("Q025", "ambient lounge tunes", "park, quiet, neutral"),
]


    df = run_semantic_eval(cases)
    print(df)




[Q001] lofi study beats focus -> mean=0.3038 best=0.5231 p@k=0.0 hallucinated=True
[Q002] instrumental concentration playlist -> mean=0.3717 best=0.6752 p@k=0.3 hallucinated=True
[Q003] chill cafe background for work -> mean=0.4948 best=0.5787 p@k=0.0 hallucinated=True
[Q004] deep focus ambient sound -> mean=0.5271 best=0.6772 p@k=0.4 hallucinated=True
[Q005] chillout jazz relax evening -> mean=0.6557 best=0.8041 p@k=0.6 hallucinated=False
[Q006] soft acoustic coffee break -> mean=0.5483 best=0.6517 p@k=0.4 hallucinated=True
[Q007] relaxing piano background -> mean=0.5681 best=0.7258 p@k=0.7 hallucinated=True
[Q008] smooth rnb unwind -> mean=0.3534 best=0.5053 p@k=0.0 hallucinated=True
[Q009] calm sleep ambient -> mean=0.6236 best=0.7464 p@k=0.6 hallucinated=False
[Q010] rain sounds for sleeping -> mean=0.6126 best=0.7862 p@k=0.6 hallucinated=False
[Q011] night meditation deep sleep -> mean=0.5362 best=0.6963 p@k=0.5 hallucinated=True
[Q012] energetic workout pop edm -> mean=0.4766 bes

### 시각화

로컬 임베딩 평가 코드로 저장한 semantic_alignment_report_local.csv를 읽는 전제

**과정(Workflow)**

1. CSV 로드 → 결측/타입 정리
2. intent_context에서 location/goal 추출(간단 파서)
3. 전체/goal별/location별 메트릭 계산
환각율(%) = hallucinated 평균
mean_sim 분포, best_sim 분포, p@k 분포
4. 단일 플롯 기준으로 히스토그램/박스플롯/막대그래프 생성 및 PNG 저장

In [36]:
# ------------------------------------------------------------
#  Semantic Alignment - Visualization (matplotlib only)
#  by Gin / Soundscape 프로젝트
# ------------------------------------------------------------
import os, json, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# === 0) 경로 설정 ===
CSV_PATH = "semantic_alignment_report_local.csv"   # 앞 단계에서 저장한 파일

# === 1) 데이터 로드 및 전처리 ===
df = pd.read_csv(CSV_PATH)

# 타입 정리
num_cols = ["mean_sim", "best_sim", "p_at_k"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# hallucinated가 문자열('True'/'False')인 경우 bool로 변환
if df["hallucinated"].dtype == object:
    df["hallucinated"] = df["hallucinated"].astype(str).str.lower().isin(["true", "1", "yes"])

# === 2) intent_context에서 location/goal 추출 ===
# 프로젝트에서 자주 쓰는 후보어휘(필요시 추가)
LOCATIONS = ["library", "cafe", "office", "subway", "outdoor", "home", "gym", "moving", "co-working", "coworking"]
GOALS = ["focus", "relax", "sleep", "active", "meditate", "reading", "neutral"]

def extract_first_match(text, vocab):
    if not isinstance(text, str):
        return ""
    t = text.lower()
    for w in vocab:
        if re.search(rf"\b{re.escape(w)}\b", t):
            return w
    return ""

df["location"] = df["intent_context"].apply(lambda s: extract_first_match(s, LOCATIONS))
df["goal"] = df["intent_context"].apply(lambda s: extract_first_match(s, GOALS))

# === 3) 핵심 메트릭 테이블 ===
summary_overall = pd.DataFrame({
    "n_cases": [len(df)],
    "hallucination_rate": [df["hallucinated"].mean()],    # 비율(0~1)
    "mean_of_mean_sim": [df["mean_sim"].mean()],
    "mean_of_best_sim": [df["best_sim"].mean()],
    "mean_of_p_at_k": [df["p_at_k"].mean()]
})

summary_by_goal = df.groupby("goal").agg(
    n_cases=("query_id","count"),
    hallucination_rate=("hallucinated","mean"),
    mean_of_mean_sim=("mean_sim","mean"),
    mean_of_best_sim=("best_sim","mean"),
    mean_of_p_at_k=("p_at_k","mean")
).sort_values("n_cases", ascending=False).reset_index()

summary_by_loc = df.groupby("location").agg(
    n_cases=("query_id","count"),
    hallucination_rate=("hallucinated","mean"),
    mean_of_mean_sim=("mean_sim","mean"),
    mean_of_best_sim=("best_sim","mean"),
    mean_of_p_at_k=("p_at_k","mean")
).sort_values("n_cases", ascending=False).reset_index()

print("\n[Overall]")
print(summary_overall.to_string(index=False))
print("\n[By Goal]")
print(summary_by_goal.to_string(index=False))
print("\n[By Location]")
print(summary_by_loc.to_string(index=False))

# === 4) 저장 폴더 ===
os.makedirs("figs", exist_ok=True)

# === 5) 시각화 (규칙: matplotlib, 단일 플롯, 컬러 미지정) ===

# 5-1. mean_sim 분포 히스토그램
plt.figure()
df["mean_sim"].plot(kind="hist", bins=20, edgecolor="black")
plt.title("Distribution of mean_sim")
plt.xlabel("mean_sim")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("figs/01_hist_mean_sim.png", dpi=200)
plt.close()

# 5-2. best_sim 분포 히스토그램
plt.figure()
df["best_sim"].plot(kind="hist", bins=20, edgecolor="black")
plt.title("Distribution of best_sim")
plt.xlabel("best_sim")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("figs/02_hist_best_sim.png", dpi=200)
plt.close()

# 5-3. p_at_k 분포 히스토그램
plt.figure()
df["p_at_k"].plot(kind="hist", bins=20, edgecolor="black")
plt.title("Distribution of p@K")
plt.xlabel("p@K")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("figs/03_hist_p_at_k.png", dpi=200)
plt.close()

# 5-4. goal별 mean_sim 박스플롯 (샘플 수가 적으면 점검)
plt.figure()
# goal이 공백일 수 있으므로 대체
df_box = df.copy()
df_box["goal"] = df_box["goal"].replace("", "unknown")
# matplotlib boxplot은 그룹핑이 직접 안되므로 수동 준비
groups = [g["mean_sim"].dropna().values for _, g in df_box.groupby("goal")]
labels = [name for name, _ in df_box.groupby("goal")]
plt.boxplot(groups, labels=labels, showfliers=False)
plt.title("mean_sim by goal")
plt.xlabel("goal")
plt.ylabel("mean_sim")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.savefig("figs/04_box_mean_sim_by_goal.png", dpi=200)
plt.close()

# 5-5. goal별 환각율 막대그래프
plt.figure()
goal_rate = df_box.groupby("goal")["hallucinated"].mean().sort_values(ascending=False)
goal_rate.plot(kind="bar")
plt.title("Hallucination Rate by goal")
plt.xlabel("goal")
plt.ylabel("rate")
plt.tight_layout()
plt.savefig("figs/05_bar_hallu_rate_by_goal.png", dpi=200)
plt.close()

# 5-6. location별 환각율 막대그래프
plt.figure()
df_loc = df.copy()
df_loc["location"] = df_loc["location"].replace("", "unknown")
loc_rate = df_loc.groupby("location")["hallucinated"].mean().sort_values(ascending=False)
loc_rate.plot(kind="bar")
plt.title("Hallucination Rate by location")
plt.xlabel("location")
plt.ylabel("rate")
plt.tight_layout()
plt.savefig("figs/06_bar_hallu_rate_by_location.png", dpi=200)
plt.close()

# 5-7. 상관산점: mean_sim vs p@K (관계 가늠)
plt.figure()
plt.scatter(df["mean_sim"], df["p_at_k"])
plt.title("mean_sim vs p@K")
plt.xlabel("mean_sim")
plt.ylabel("p@K")
plt.tight_layout()
plt.savefig("figs/07_scatter_mean_vs_patk.png", dpi=200)
plt.close()

print("\n 생성된 이미지: figs/01~07_*.png")



[Overall]
 n_cases  hallucination_rate  mean_of_mean_sim  mean_of_best_sim  mean_of_p_at_k
      25                0.88          0.469292          0.652244           0.268

[By Goal]
   goal  n_cases  hallucination_rate  mean_of_mean_sim  mean_of_best_sim  mean_of_p_at_k
               6            1.000000          0.419767          0.603467        0.100000
 active        4            1.000000          0.478550          0.703325        0.250000
  focus        4            1.000000          0.424350          0.613550        0.175000
neutral        4            1.000000          0.426050          0.625500        0.250000
  relax        4            0.750000          0.531375          0.671725        0.425000
  sleep        3            0.333333          0.590800          0.742967        0.566667

[By Location]
location  n_cases  hallucination_rate  mean_of_mean_sim  mean_of_best_sim  mean_of_p_at_k
    home        7            0.714286          0.512557          0.680214        0.37142

C:\Users\seonu\AppData\Local\Temp\ipykernel_28648\3785327211.py:117: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(groups, labels=labels, showfliers=False)



 생성된 이미지: figs/01~07_*.png


In [37]:
# === 6) 전체 환각율(Overall Hallucination Rate) ===
overall_rate = float(df["hallucinated"].mean()) if len(df) else 0.0
overall_pct = round(overall_rate * 100, 2)

print("\n[Overall Hallucination]")
print(f"- 전체 케이스: {len(df)}")
print(f"- 환각 케이스 수: {int(df['hallucinated'].sum())}")
print(f"- 전체 환각율: {overall_pct}%")

# 6-1. 전체 환각율 막대그래프 (1개 플롯)
plt.figure()
counts = [int((~df['hallucinated']).sum()), int(df['hallucinated'].sum())]
labels = ["aligned", "hallucinated"]
plt.bar(labels, counts)
plt.title(f"Overall Hallucination Rate ({overall_pct}%)")
plt.xlabel("class")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("figs/08_bar_overall_hallucination.png", dpi=200)
plt.close()

# 6-2. 전체 환각율 도넛(파이) 차트 (1개 플롯)
plt.figure()
# 파이 차트
plt.pie(counts, labels=labels, autopct="%1.1f%%", startangle=90)
# 도넛 형태를 위해 가운데 원 추가
centre_circle = plt.Circle((0, 0), 0.70, fc="white")
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title(f"Overall Hallucination (N={len(df)})")
plt.tight_layout()
plt.savefig("figs/09_donut_overall_hallucination.png", dpi=200)
plt.close()

print(" 생성된 이미지 추가: figs/08_bar_overall_hallucination.png, figs/09_donut_overall_hallucination.png")



[Overall Hallucination]
- 전체 케이스: 25
- 환각 케이스 수: 22
- 전체 환각율: 88.0%
 생성된 이미지 추가: figs/08_bar_overall_hallucination.png, figs/09_donut_overall_hallucination.png


In [38]:
df[['gpt_query','mean_sim','best_sim','p_at_k','hallucinated']].sort_values('mean_sim')


,gpt_query,mean_sim,best_sim,p_at_k,hallucinated
23,background pop hits,0.2857,0.4684,0.0,True
0,lofi study beats focus,0.3038,0.5231,0.0,True
15,heavy metal anger release,0.3270,0.6001,0.1,True
18,comforting acoustic ballads,0.3472,0.5549,0.0,True
7,smooth rnb unwind,0.3534,0.5053,0.0,True
1,instrumental concentration playlist,0.3717,0.6752,0.3,True
21,daily mix chill pop,0.3734,0.6476,0.1,True
14,driving upbeat hits,0.4106,0.7532,0.1,True
20,soothing piano for heartbreak,0.4157,0.6808,0.2,True
19,warm indie songs for comfort,0.4228,0.6013,0.1,True


위 결과는 GPT가 생성한 검색 키워드가 Spotify 검색 결과와 의미적으로 얼마나 일치하는지를 정량화한 평가셋임

*칼럼 설명*
| 열 이름               | 데이터 타입      | 설명                                                                                                                                                                                                                        |
| ------------------ | ----------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **query_id**       | `str`       | 평가 대상 쿼리의 식별자. CSV 평가셋의 고유 번호(ID). 예: `"Q001"`.                                                                                                                                                                           |
| **gpt_query**      | `str`       | GPT가 생성한 **Spotify 검색용 키워드**. 실제 Spotify Search API의 입력값이자 환각 검증의 핵심 대상.                                                                                                                                                  |
| **intent_context** | `str`       | 쿼리의 **맥락(Context)** — 예를 들어 `"library, quiet, focus"` 같은 환경·목표 정보. 모델의 의도 파악을 돕는 설명.                                                                                                                                      |
| **mean_sim**       | `float`     | 검색 결과 상위 K개(기본 10개)의 **평균 코사인 유사도**. <br> GPT 쿼리와 Spotify 결과 문서 간 **의미적 일치 정도(0~1)** 의 평균값.                                                                                                                               |
| **best_sim**       | `float`     | 상위 K 중 **가장 높은 코사인 유사도(Top-1)**. <br> 가장 잘 맞는 결과가 쿼리와 얼마나 일치했는지 측정.                                                                                                                                                       |
| **p_at_k**         | `float`     | Precision@K 지표. 상위 K 결과 중 **유사도 ≥ 임계값(기본 0.6)** 인 비율. <br> 예: 10개 중 7개가 0.6 이상이면 `p_at_k = 0.7`.                                                                                                                          |
| **hallucinated**   | `bool`      | 환각 여부(True/False). <br> 평균 유사도·P@K·최고 유사도 중 하나라도 기준치 미달이면 `True`로 표시. <br> 즉, GPT 키워드가 실제 결과와 **맥락적으로 불일치**했음을 의미.                                                                                                        |
| **top_examples**   | `str(JSON)` | 상위 3개 Spotify 검색 결과의 **요약 정보 리스트**. <br> 각 항목에 `rank`, `id`, `type(track/playlist)`, `sim(유사도)`, `text(요약 문장)`이 포함됨. <br> 예: <br>`[{"rank":1,"id":"7Ht6d","type":"track","sim":0.8231,"text":"Lofi Study Beats by ..."}]` |


*지표 임계값*
| 지표                      | 해석 기준               | 판단                                     |
| ----------------------- | ------------------- | -------------------------------------- |
| **mean_sim ≥ 0.60**     | 쿼리와 결과 전반이 의미적으로 일관 | 정상                                     |
| **best_sim < 0.62**     | 상위 결과도 불일치          | 환각 의심                                  |
| **p_at_k < 0.3**        | 유의미한 결과 비율이 낮음      | 환각 가능성 높음                              |
| **hallucinated = True** | 위 조건 중 하나라도 위반됨     | “GPT가 잘못된 키워드 생성” 또는 “Spotify 결과가 엉뚱함” |


*해석 예시*
| query_id | gpt_query                 | mean_sim | p_at_k | hallucinated | 해석                             |
| -------- | ------------------------- | -------- | ------ | ------------ | ------------------------------ |
| Q001     | lofi study beats focus    | 0.79     | 0.8    | False        | GPT 쿼리와 Spotify 결과가 잘 일치 (정상)  |
| Q002     | energetic workout pop edm | 0.41     | 0.2    | True         | 결과들이 의도(활동적 운동)와 의미적으로 멀음 → 환각 |
| Q003     | calm sleep ambient        | 0.83     | 0.9    | False        | 완벽히 일치 (수면·앰비언트 관련)            |


---